<img align="right" src="tf-small.png"/>

# Search

Do we need search in TF, like MQL?

Yes, it is convenient to have a more declarative way of getting a set of interesting nodes to work with.

# Query syntax

## General
We have these kinds of lines:

* white-space lines (everywhere allowed, will be always ignored)
* relation line: **name operator name**.
  Indents and spacing are ignored, but there must be space around the operator.
* atom line: **indent otype features**. The indent is significant.
* feature line: **features**. Indent is not significant. Only allowed after an atom line or after
  a feature line. 
  Feature lines are continuations of the features of an atom, handy in those cases where the
  features occupy a lot of space.

## Features

A white-space separated list of *key*=*values*.

* there may be no space around the `=`.
* *key* must be a feature name that exists in the dataset.
  If it is not yet loaded, it will be loaded.
* *values* must be a `|` separated list of feature values, no quotes.
  No spaces around the `|`.
  If you need a space or `|` or `\\` in a value, escape it by a `\\`.
  Escape tabs and newlines as `\\t` and `\\n`.

## Operators

### Node comparison
* `=`: is equal (meaning the same node, a clause and a verse that occupy the same slots are still unequal)
* `#`: is unequal (meaning a different node, a clause and a verse that occupy the same slots are still unequal)
* `<` `>`: before and after (in the *canonical ordering*)

### Slot comparison
* `==`: occupy the same slots (identical slot sets)
* `&&`: overlap (the intersection of both slot sets is not empty)
* `##`: occupy different slots (but they may overlap, the set of slots of the two are different as sets)
* `||`: occupy disjoint slots (no slot occupied by the one is also occupied by the other)
* `[[` `]]`: embeds and contains (slot set inclusion, in both directions)
* `<<` `>>`: before and after (with respect to the slots occupied: left ends before right starts and vv)

### Edge features
* `-`*name*`>` `<`*name*`-`: connected by the edge feature *name*, in both directions

# Class Query 

We receive the nodes and edges of a query, give the nodes a number,
and replace the *from*- and *to*-nodes
of the edges by the numbers of those nodes.

We also build a list of otypes of the nodes, in the same order as the nodes themselves.

Then we are ready to perform all kinds of operations with this query.

## Atoms

The first stage of running a query is to run the individual nodes as filters in their associated
object types.

A node is specified by an `otype` and a features dict.
The features specifiy any number of features, with an allowed value or set of allowed values for each of them.

Running an atom means to filter the set of all nodes in its `otype` by means of its `features`.

The result list is delivered in a list of result lists, corresponding to the nodes list.

## Edges

Once the atoms have done their work, it is time to work out the constraints posed by edges.

An edge from query node `nF` to query node `nT` means
* that the text nodes in the result of `nF` should embed a text node in the result of `nT`,
or equivalently,
* that the text nodes in the result of `nT` should be embedded in a text node in the result of `nF`.

This will reduce the amount of nodes in both `nF` and `nT`.

### Note
> In general, one pass over all edges will not be enough.
We have to repeat the process until nothing changes anymore.

## Strategy

The challenge is now to run the edges in an optimal sequence.

The basic intuition is this.

* some query nodes filter strongly, others hardly, i.e. some atom results are small compared to the total
  number of nodes in their otype, other atom results are nearly as big as the total otype.
* if an edge connects a strongly filtering node with a weakly filtering node, we expect a big reduction
* if we work within the strongest filtering query nodes, we do not have to do much work and when we reach the
  weaker filters, they will decrease rapidly
* so we postpone to touch the bigger sets as long as possible, and when we touch them, they are expected to decrease
  quickly

We are going to rank query nodes by how strong they have filtered their otype so far.

* let $o_n$ be the otype associated with query node $n$
* let $r_n$ be the current result set associated with query node $n$


Then the **query fraction** $q(n)$ is the a proportion between
the number of text nodes in the current result:
and
the total number of text nodes in the otype:

$$q(n) = {{|r_n|}\over {|o_n|}}$$

Then we rank the edges by combined query fraction of the nodes:

$$ r(f,t) = q(f)^2 + q(t)^2$$

By squaring both query fractions, we strongly give precedence to edges involving few results.

## Showing

We want to see intermediate yarns.

In [265]:
class Query(object):
    def __init__(self, rawNodes, rawEdges):
        self.good = True
        otypes = []
        features = []
        edges = []
        nodeIndex = {}
        for (q, node) in enumerate(rawNodes):
            fNode = freeze(node)
            nodeIndex[fNode] = q
            otypes.append(node[0])
            features.append(node[1])
        for (nodeFrom, relation, nodeTo) in rawEdges:
            (fNodeFrom, fNodeTo) = (freeze(nodeFrom), freeze(nodeTo))
            if fNodeFrom not in nodeIndex:
                error('"from"-node  in edge {} is not found: {}'.format(i, nodeFrom))
                self.good = False
                continue
            if fNodeTo not in nodeIndex:
                error('"to"-node in edge {} is not a node: {}'.format(i, nodeTo))
                self.good = False
                continue
            fromI = nodeIndex[fNodeFrom]
            toI = nodeIndex[fNodeTo]
            edges.append((fromI, relation, toI))
        self.otypes = otypes
        self.features = features
        self.qnodes = list(range(len(otypes)))
        self.edges = edges
        self.yarns = {}
        self.spreads = {}
        self.spreadsC = {}
        self.uptodate = {}
        self.connectedness()
        self.verbose = None
    
    def connectedness(self):
        componentIndex = dict(((q, {q}) for q in range(len(self.otypes))))
        for (f, rela, t) in self.edges:
            if f != t:
                componentIndex[f] |= componentIndex[t]
                for u in componentIndex[f] - {f}:
                    componentIndex[u] = componentIndex[f]
        components = sorted(set(frozenset(c) for c in componentIndex.values()))
        componentIndex = {}
        for c in components:
            for q in c:
                componentIndex[q] = c
        componentEdges = {}
        for (e, (f, rela, t)) in enumerate(self.edges):
            c = componentIndex[f]
            componentEdges.setdefault(c, []).append(e)
        self.components = []
        for c in components:
            self.components.append([
                sorted(c),
                componentEdges[c]
            ])
        indent(level=0)
        lComps = len(self.components)
        if lComps == 0:
            error('Query without instructions. Tell me what to look for.')
            self.good = False
        elif lComps > 1:
            error('More than one connected components ({}):'.format(len(self.components)))
            error('Either run the subqueries one by one, or connect the components by a relation',tm=False)
            self.good = False
        else:
            info('Query OK: {} query nodes and {} query edges'.format(len(self.qnodes), len(self.edges)))
                
    def showNode(self, q):
        info('node {:>2}-{:<13} ({:>6} in yarn)'.format(
            q, self.otypes[q], len(self.yarns[q]),
        ), tm=False)


    def showEdge(self, e, dir):
        otypes = self.otypes
        es = self.edges
        spreads = self.spreads
        spreadsC = self.spreadsC
        (f, rela, t) = es[e]
        if dir == -1: (f, t) = (t, f)
        info('edge {:>2}-{:<13} ={:>2}=> {:>2}-{:<13} @{:8.3g}'.format(
            f, otypes[f], dir, t, otypes[t],
            spreads.get(e, '?') if dir == 1 else spreadsC.get(e, '?'),
        ), tm=False)

    def showYarns(self):
        indent(level=0)
        indent(level=1)
        for q in self.qnodes:
            self.showNode(q)

    def showCandidate(self, title, qs, esO):
        info('{} with {} nodes and {} edges'.format(title, len(qs), len(esO)), tm=False)
        for q in qs: self.showNode(q)
        for e in esO: self.showEdge(*e)

    def spinAtom(self, q):
        otype = self.otypes[q]
        features = self.features[q]
        featureList = sorted(features.items())
        yarn = set()
        for n in F.otype.s(otype):
            good = True
            for (ft, val) in featureList:
                fval = Fs(ft).v(n)
                if type(val) is str or type(val) is int:
                    if fval != val:
                        good = False
                        break
                else:
                    if fval not in val:
                        good = False
                        break
            if good: yarn.add(n)
        self.yarns[q] = yarn

    def spinAtoms(self):
        qs = self.qnodes
        indent(level=0)
        info('Spinning principal yarn for {} atoms'.format(len(qs)))
        for q in qs:
            self.spinAtom(q)
        self.showYarns()

    def estimateSpreads(self, both=False):
        otypes = self.otypes
        es = self.edges
        yarns = self.yarns

        self.spreadsC = {}
        self.spreads = {}

        for (e, (f, rela, t)) in enumerate(es):
            tasks = [(f, rela, t, self.spreads)]
            if both:
                tasks.append((t, converse[rela], f, self.spreadsC))
            for (tf, trela, tt, dest) in tasks:
                yarnF = list(yarns[tf])
                yarnT = yarns[tt]
                function = types.FunctionType
                totalSpread = 0
                if len(yarnF) < 100:
                    tries = yarnF
                else:
                    tries = set(yarnF[randrange(len(yarnF))] for n in range(100))
                if len(tries) == 0:
                    dest[e] = 0
                else:
                    r = relations[trela](otypes[tf], otypes[tt])
                    for n in tries:
                        image = r(n)
                        if type(image) is function:
                            nRela = set(m for m in yarnT if image(m))
                            totalSpread += len(nRela)
                        else:
                            nRela = set(image) & yarnT
                            totalSpread += len(nRela)
                    dest[e] = totalSpread / len(tries)
        indent(level=0)
        info('Estimated spreads:', tm=False)
        indent(level=1)
        for e in range(len(es)):
            self.showEdge(e, 1)
            if both: self.showEdge(e, -1)

    def chooseEdge(self):
        yarnFractionNode = {}
        qs = self.qnodes
        es = self.edges
        otypes = self.otypes
        spreads = self.spreads
        for q in qs:
            otype = otypes[q]
            (begin, end) = F.otype.sInterval(otype)
            nOtype = 1 + end - begin
            nYarn = len(self.yarns[q])
            yf = nYarn / nOtype
            yarnFractionNode[q] = yf * yf
        yarnFractionEdge = {}
        for (e, (f, rela, t)) in enumerate(es):
            if self.uptodate[e]: continue
            yarnFractionEdge[e] = yarnFractionNode[f] + yarnFractionNode[t] + spreads[e]
        indent(level=0)
        firstEdge = sorted(yarnFractionEdge.items(), key=lambda x: x[1])[0][0]
        return firstEdge
            
    def spinEdge(self, e):
        otypes = self.otypes
        yarns = self.yarns
        qs = self.qnodes
        es = self.edges
        uptodate = self.uptodate
        
        (f, rela, t) = es[e]
        yarnF = yarns[f]
        yarnT = yarns[t]
        indent(level=1, reset=True)
        self.showEdge(e, 1)
        self.showNode(f)
        self.showNode(t)

        newYarnF = set()
        newYarnT = set()
        function = types.FunctionType
        
        r = relations[rela](otypes[f], otypes[t])
        for n in yarnF:
            image = r(n)
            if type(image) is function:
                nRela = set(m for m in yarnT if image(m))
                if len(nRela):
                    newYarnT |= nRela
                    newYarnF.add(n)
            else:
                nRela = set(image) & yarnT
                if len(nRela):
                    newYarnT |= nRela
                    newYarnF.add(n)

        affectedF = len(newYarnF) != len(yarns[f])
        affectedT = len(newYarnT) != len(yarns[t])

        uptodate[e] = True
        for (oe, (of, orela, ot)) in enumerate(es):
            if oe == e: continue
            if (affectedF and f in {of, ot}) or (affectedT and t in {of, ot}):
                self.uptodate[oe] = False
        self.yarns[f] = newYarnF
        self.yarns[t] = newYarnT      
        self.showNode(f)
        self.showNode(t)

    def spinEdges(self):
        qs = self.qnodes
        es = self.edges
        yarns = self.yarns
        uptodate = self.uptodate

        indent(level=0)
        info('Spinning with {} edges'.format(len(es)))
        for e in range(len(es)):
            uptodate[e] = False
        it = 0
        while True:
            if min(len(yarns[q]) for q in qs) == 0: break
            if reduce(
                lambda y,z: y and z, 
                (uptodate[e] for e in range(len(es))),
                True,
            ): break
            e = self.chooseEdge()
            self.spinEdge(e)
            it += 1
        indent(level=0)
        info('Done {} iterations'.format(it))
        self.showYarns()

    def makeS1(self):
        qs = self.qnodes
        es = self.edges
        s1Edges = []
        for (e, (f, rela, t)) in enumerate(es):
            if self.spreads[e] <= 1:
                s1Edges.append((e, 1))
            if self.spreadsC[e] <= 1:
                s1Edges.append((e, -1))
        # s1Edges contain all edges with spread <= 1, or whose converse has spread <= 1
        # now we want to build the largest graph with the original nodes and these edges,
        # such that you can walk from a starting point over directed s1 edges to every other point
        # we initialize candidate graphs: for each node: singletons graph, no edges.
        candidates = []
        # add s1 edges and nodes to all candidates
        for q in qs: 
            cnodes = {q}
            cedges = set()
            cedgesOrder = []
            while True:
                added = False
                for (e, dir) in s1Edges:
                    (f, rela, t) = es[e]
                    if dir == -1: (t,f) = (f,t)
                    if f in cnodes:
                        if t not in cnodes:
                            cnodes.add(t)
                            added = True
                        if (e, dir) not in cedges:
                            cedges.add((e, dir))
                            cedgesOrder.append((e, dir))
                            added = True
                if not added: break
            candidates.append((cnodes, cedgesOrder))

        # pick the biggest graph (nodes and edges count for 1)
        startS1 = sorted(candidates, key=lambda x: len(x[0])+len(x[1]))[-1]
        # remove spurious edges: if we have both the 1 and -1 version of an edge,
        # we can leave out the one that we encounter in the second place
        newCedges = set()
        newCedgesOrder = []
        for (e, dir) in startS1[1]:
            if e not in newCedges:
                newCedgesOrder.append((e, dir))
                newCedges.add(e)
        startS1 = (startS1[0], newCedgesOrder)
        self.showCandidate('startS1', *startS1)
        return startS1

    def stitchPlan(self, startS1):
        es = self.edges

        newNodes = startS1[0]
        newEdges = startS1[1]
        doneEdges = set(e[0] for e in newEdges)
        # we add all edges that are not yet in our startS1.
        # we add them two-fold: also with converse, and we sort the result by spread
        # then we start a big loop:
        # in every iteration, we take the edge with smallest spread that can be connected
        # to the graph under construction
        # then we start a new iteration, because the graph has grown, and and new edges might
        # have become connectable by that

        remainingEdges = set()
        for e in range(len(es)):
            if e not in doneEdges:
                remainingEdges.add((e, 1))
                remainingEdges.add((e, -1))
        remainingEdgesO = sorted(
            remainingEdges,
            key=lambda e: self.spreads[e[0]] if e[1] == 1 else self.spreadsC[e[0]],
        )

        info('Making a Plan. {} untreated edges so far:'.format(
                len(set(e[0] for e in remainingEdgesO))),
            tm=False)
        for e in remainingEdgesO: self.showEdge(*e)

        while True:
            added = False
            for (e, dir) in remainingEdgesO:
                if e in doneEdges: continue
                (f, rela, t) = es[e]
                if dir == -1: (f, t) = (t, f)
                if f in newNodes:
                    newNodes.add(t)
                    newEdges.append((e, dir))
                    doneEdges.add(e)
                    added = True
                    break
            if not added: break
        # conjecture: we have all edges and all nodes now
        # reason: we work in a connected component, so all nodes are reachable
        # by edges or inverses
        self.showCandidate('Plan', newNodes, newEdges)
        return (newNodes, newEdges)
        
    def stitch(self):
        qs = self.qnodes
        es = self.edges
        yarns = self.yarns

        self.estimateSpreads(both=True)
        indent(level=0)
        info('Stitching {} yarns with {} stitches. Total yarn length {}'.format(
            len(qs), len(es),
            sum(len(yarns[q]) for q in qs)
        ))
        startS1 = self.makeS1()
        plan = self.stitchPlan(startS1)
        deliver = self.stitchResults(plan)
        indent(level=0, verbose=-2)
        info('Stitching done')
        
    def stitchResults(self, plan):
        otypes = self.otypes
        es = self.edges
        yarns = self.yarns
        function = types.FunctionType

        planEdges = plan[1]
        if len(planEdges) == 0:
            # no edges, hence a single node (because of connectedness,
            # hence we must deliver everything of its yarn
            yarn = yarns[0]
            def deliver():
                for n in yarn: yield n
            return deliver
                
        (firstE, firstDir) = planEdges[0]
        (firstF, firstRela, firstT) = es[firstE]
        if firstDir == -1:
            (firstF, firstT) = (firstT, firstF)
            firstRela = converse[firstRela]
        firstR = relations[firstRela](otypes[firstF], otypes[firstT])
        yarnFirstF = yarns[firstF]
        yarnFirstT = yarns[firstT]

        def deliver():
            def stitchOn(stitch, ie):
                if ie >= len(planEdges):
                    yield stitch
                    return
                (e, dir) = planEdges[ie]
                (f, rela, t) = es[e]
                if dir == -1:
                    (f, t) = (t, f)
                    rela = converse[rela]
                r = relations[rela](otypes[f], otypes[t])
                yarnF = yarns[f]
                yarnT = yarns[t]
                n = stitch[-1]
                image = r(n)
                if type(image) is function:
                    nRela = set(m for m in yarnT if image(m))
                else:
                    nRela = set(image) & yarnT
                for m in nRela:
                    for s in stitchOn(stitch+(m,), ie+1):
                        yield s
            
            for n in yarnFirstF:
                image = firstR(n)
                if type(image) is function:
                    nRela = set(m for m in yarnFirstT if image(m))
                else:
                    nRela = set(image) & yarnFirstT
                if len(nRela) == 0: # no stitch here
                    continue
                for m in nRela:
                    for s in stitchOn((n, m), 1):
                        yield s

        self.deliver = deliver
                   
    def runQuery(self):
        if not self.good: return None
        indent(level=0, reset=True)
        self.spinAtoms()
        self.estimateSpreads()
        self.spinEdges()
        self.stitch()
        return self.deliver

But should it be MQL?

Experience shows that MQL may give you a very good first try, 
until you realize that you may not have queried for all cases.
You forgot to query for some elements in a different order.
You have not reckoned with gaps.
And the query does not give you interesting things from the context with the results.
Also, MQL does not work nicely with object types that are scattered through other object types, such as the
[*lexeme*](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/otype.html)
type.

Look in SHEBANQ for examples how unwieldy MQL queries may become.

Here is a good example:

[Dirk Roorda: Yesh](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=556)

```
select all objects where
[book [chapter [verse
[clause
    [clause_atom
        [phrase
            [phrase_atom
                [word focus lex="JC/" OR lex=">JN/"]
            ]
        ]
    ]
]
]]]
```

Well, this is not too complicated, but the query misses results.
See [here](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_mql.html)
to see what would be needed to make it right.

Also, I wonder: do we want a new language?
Suppose we make a TFQL, then we need a parser for it,
we need to define a syntax, we need to refine the syntax, update the parser, etc.
It will become a cumbersome straight-jacket.

In our case, we do not have the requirement that non-coders should be able to use TFQL in a stand-alone manner.

On the contrary, TFQL should live in a programming environment, and we can take advantage of that.

Here are initial thought for **tfQuery**, a query *mechanism* inside TF, not a *language*.

* tfQuery defines queries as data structures in Python, more precisely: as a graph
* it does not matter how you build up a query, tfQuery processses the value of a datastructure
  that you pass to it. The surface syntax will not be seen by tfQuery
* a query is a graph representation where the nodes are things like
  
  `('phrase', dict(det='und'))`
  
  or
  
  `('word', dict(sp='verb', gn='f', ps='3f'))`

* the edges specify relations between the nodes, like: *is contained in*, *follows*,
  *precedes*
  
In MQL you also specify a graph, by means of a template, but this template forces you to *overspecify*: the template often implies more constraints then you really want.

So how do we specify edges? As constraints.

Let us formulate a query for

* clauses that are object clauses
* containing two phrases (both undetermined)
* one of which contains a verb in the third person feminine
* and the other phrase contains a feminine, plural noun

In MQL

```
[clause rela='Objc'
    [phrase det='und'
        [word sp='verb' AND gn='f' AND ps='p3']
    ]
    [phrase det='und'
        [word sp='subs' AND gn='f' AND nu='pl']
    ]
]
```

Here is how we are going to do it,
and note that we are going to write executable code!

In [1]:
c = ('clause', dict(rela='Objc'))
p1 = ('phrase', dict(det='und'))
p2 = ('phrase', dict(det='und'))
w1 = ('word', dict(sp='verb', gn='f', ps='p3'))
w2 = ('word', dict(sp='subs', gn='f', nu='pl'))

In [2]:
nodes = [c, p1, p2, w1, w2]
edges = [
    (c, [p1,p2]),
    (p1, [w1]),
    (p2, [w2]),
    (p1, p2),
]

query = (nodes, edges)

An edge of like `(x, [y,z])` means that `y` and `z` are embedded in `x`, but does not mean
that `y` comes before `z`.

An edge like `(x, y)` means that `x` comes before `y`.

## Increased flexibility

Note that it is very easy to remove the `(p1, p2)` condition, which states that the first
phrase comes before the second one.

If we wanted to do that in MQL, the query would become:

```
[clause rela='Objc'
    [phrase det='und'
        [word sp='verb' AND gn='f' AND ps='p3']
    ]
    [phrase det='und'
        [word sp='subs' AND gn='f' AND nu='pl']
    ]
    OR
    [phrase det='und'
        [word sp='subs' AND gn='f' AND nu='pl']
    ]
    [phrase det='und'
        [word sp='verb' AND gn='f' AND ps='p3']
    ]
]
```

This goes quickly out of hand, see e.g.
[Dirk Roorda: Object clauses of verbless mothers](https://shebanq.ancient-data.org/hebrew/query?id=984) and accompanying
[notebook](https://shebanq.ancient-data.org/shebanq/static/docs/tools/shebanq/VerblessMothers.html)

## Query results

What should we return as query results?
Do we want every instantiation of the nodes that satisfy the criteria?

That can become overwhelming. 
If for example you search for a word in a book, an other word in the same book, and a third word in the same book without further constraints, then for a book with 10,000 words you'll get 10,000 * 10,000 * 10,000 results or 1 Tera results, which is, even for a computer, a bit much.

This is why Ulrik invented the sheaf.

Our way of solving this problem could be like this:

* we return a collection of node lists: for each node in the query we return the 
  corresponding node list;
* these lists consist of TF nodes which are guaranteed to occur in at least one
  instantatiation of the whole graph;
* we return nothing else.

It is up to the user to pick one of these nodesets and to further process them.
If he needs needs context around the result nodes, he can easily draw the info from there.

It is even possible to generate the code to get full results from the original query graph.

So the real result is two things:

* a collection of node lists
* a function to look up other result nodes in the context of a given result node.

## Implementation

How could we implement this search efficiently?

First idea:

* build for each query graph node
  (which corresponds to a local feature condition on an object)
  the set of nodes that satisfy the condition.
  This is the easy part:
  A single walk over all nodes could construct these sets in one go, in a fraction
  of a second;
* then work through all edges, where every edge is an instruction to weed out non-results
  from the earlier obtained sets.
  
How would that work, filtering along an edge in the graph?

Suppose there is an edge from node1 to node2 (in the query graph).
This edge specifies a relationship between nodes in the result nodeset of n1 and nodes 
in the result nodeset of n2.

In English: such an edge says: hey TF node in result set of n1: 
do you have a parent (or child, or older brother or younger sister)
that occurs in the result of n2?

If so: you can stay. If not: you're OUT.
So this weeds out TF nodes from the result set of n1.

But we can also reduce the nodes in the result set of n2.
Every TF node in the result set of n2 that does not figure as the parent
(or child or brother/sister) of a TF node in the result set of n1 is also out.

In this way we can take every edge, one by one, and perform the filtering.
This is also a fast operation, provided we can make the elementary relationship checks
quickly. (And we can, in TF, thanks to precomputing).

When we have done all edges, we probably have to iterate again over all edges.

Because an edge between n2 and n3 could have weeded out additional TF nodes from the 
result set of n2, and this influences the validity of the TF nodes in the result set of n1.

Probably we have to repeat until the result sets do not change anymore.

Or we can find a way to order the edges, so that we can do them in one or two passes.

Maybe we need a bit of math here.

My gut feeling is that this is all very doable and that it corresponds to people's query
needs.

# Algorithm

We specify our algorithm in a strategy-independent way and prove the correctness of it.
In order to do that, we need a bit of language.

## Terminology

### Query graph
A query is a graph of query nodes and query edges.
We refer to the query nodes as
$$q_1, ... , q_n$$
and to the query edges as
$$e_1, ... , e_k$$
where every
$$e_j = (R_j, q_f, q_t)$$
for some $f$ and $t$ in $1, ..., n$, 
where $R_j$ is a relationship between text nodes.
That means, if $s$ and $u$ are text nodes, $sR_ju$ is either *true* or *false*.

### Query node
A query node $q_i$ consists of an
[otype](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/otype)
of text nodes and a 
[features](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_overview.html)
dict.
The features dict specifies any number of data features, with an allowed value or set of allowed values for each of them.

### Wool and yarn
The set of all text nodes of an
[otype](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/otype)
is called the *wool* of that otype.

The wool of a query node, is the wool of the otype that is associated with that query
node.

If we filter the wool of a node by some process, we call the result set a *yarn*
of that query node.

The *principal* yarn of a query node consists of the wool of the query node,
filtered by the features dict of the query node.

### Spinning and proper yarns
If we have an edge $e_j = (R_j, q_f, q_t)$ and if we have yarns $y_f, y_t$ for
$q_f, $q_t, then we *turn* the edge $e_j$ 
to *spin* the yarns $y_f$, $y_t$ further.

The result of this spinning are two new yarns, $z_f, z_t$, obtained as follows:

* we iterate through the text nodes $s$ in yarn $y_i$:
  * we compute the set of text nodes $u$ in $y_j$ such that $sRu$ holds;
  * if this set is non-empty, we:
    * add those $u$ text nodes to $z_j$
    * add the $s$ text node to $z_i$
  Otherwise we do not add nodes to $z_i$ nor to $z_j$.

So spinning means thinning out the yarns of two query nodes that are connected to a
query edge, in such a way, that afterwards the relationship that is associated
with the edge, can be realized for every text node in both yarns.
In other words: after spinning, every text node in a yarn involved, has at least one
counterpart text node in the other yarn such that both are in relationship which each other.
That is, the relationship belonging to the edge.

So, every *spinning* action is caused by a *turn* of the edge, which acts as the spinning
wheel.

A *proper yarn* of a query node is either its *principal yarn* or any yarn that can be spun 
from an other proper yarn.
So if we start with the principal yarn of a query node, and start spinning, we produce
proper yarns for that query node.

### Stitching results

A *result* of a query is a list of text nodes
$$s_1, ... , s_n$$
satisfying the following conditions:

* for every $i$ in $1, ... , n$: $s_i$ is in the principal yarn of $q_i$
* for every $e_j = (R_j, q_f, q_t)$: $s_fRs_t$

If this is the case, we call text node $s_i$ a *stitch* for query node $q_i$. 

A result can be seen as a stitching together of the principal yarns of the query nodes,
in such a way, that all constraints specified by the query edges are satisfied.
The stitches are those nodes in the text, one for each yarn, that together constitute
a result.

### Overspun and underspun yards

If a yarn of a query node has become so thin, that it fails to contain stitches of
of the result of the query,
we call the yarn *overspun*.

If a yarn of a query node contains nodes that are not stitches of any result of the query,
we call the yarn *underspun*.

### Employ the terminology

We can now use this fabric language to formulate our query algorithm and prove essential properties of it.

## Initial

Here is the preparation for running a sequence of iterations.
In those iterations we *turn* the edges to *spin* yarns.

First of all, for every query node, we collect its wool, and spin its principal yarn.

> we compute edge constraints and filter the result sets of text nodes.

> we collect all text nodes of the otype associated with the query node
(the wool), and we apply the criteria defined by the features of the query node
(from wool to principal yarn).

We give every edge a the status: *not up-to-date*.
The intention is to give an edge the status *up-to-date* if it has just been turned,
and no turnings of other edges have since interfered with the yarns involved,
so the yarns still reflect the effects of turning the edge.

Initially, no edge has been turned, so every edge starts out as not up-to-date.


## Iterations
When query edges are computed, yarns are spun,
some edges become up-to-date, others get *not* up-to-date.

* we select a query edge by means of some strategy,
* we turn the edge,
* we set the status of the edge to up-to-date,
* for each of both query nodes of the edge:
  * if its yarn has changed:
    * for each query edge leading to or from such a query node:
      * we set its status to not up-to-date

## Final
Here is when we stop turning the edges.
We stop when continuing does not make sense anymore, and that happens if one
of the following conditions occur. 
Whether we have correct and complete results if we stop, is something that
remains to be seen.
We'll prove it later.

### Stop on empty yarn
If a yarn becomes empty, we can stop: there are no results at all. 
The combined turning of the edges has spun the yard into nothing: the thread has broken.
Every yarn must be part of the result, so if a yarn breaks, the result is gone.

> The constraints expressed by the query edges are such, that one of the query nodes
cannot be instantiated by a text node. Since every query result must instantiate all
query nodes by a text node, we do not have a result.

### Stop on no change
If all edges are up-to-date, it does not make sense to turn edges anymore.
So we stop.

> Turning an up-to-date edge means recomputing the constraints posed by that edge
on node sets on which it has already been computed. There will be no effect.

## Correctness and completeness

The main questions are: 
* does the computation stop at all in all cases
* and when it stops, have we got what we want?

This fundamental question can be split into lesser ones.

### Note about strategy
The validity of the results delivered should not dependent on
the strategy of selecting edges for computation. 
But it is easy to come up with a stupid strategy that will not terminate
or not arrive at desired results.
For example, if we always select the first edge,
it is clear that this is in general a very unreliable strategy.

So we suppose that our strategy of edge selection for computation satisfies a minimal
sanity requirement. 
We can formulate that exactly:

A strategy is *thorough* if it always selects a non-up-to-date edge if there is one.

This is really a minimal criterion, since selecting an up-to-date edge is always a waste
of time.
And if there are no up-to-date edges, it is time to stop anyway.

### Note on results
The algorithm as defined above does not deliver a list of *results*, 
results being *stitches*. 
It only delivers the yarns to be stitched.

> The algorithm yields a list of text node sets
instantiations of all query nodes by text nodes in such a way that all query edges
are satisfied. Which combinations of these text nodes constitute results, remains to 
be established.

Instead, we assert that after the process, every yarn is *fully spun*, so
none of them is *underspun* and none of them is *overspun*.

> The claim is that every node set has only nodes that occur in the real results, and that
all nodes that occur in the real results, do occur in the proper node sets.

## Lemma 1: Termination
**Under every *thorough* strategy, the iteration always terminates.**

**Proof:**

If an edge is turned, no yarn becomes longer. It is possible, however, that no yarn
shrinks.

For every turn of an edge there are two cases to consider:

a. No yarn becomes shorter:
   Then the amount of up-to-date edges increases by one.
   This is so, because the recently turned edge becomes up-to-date, and no other edges    
   become non-up-to-date.
b. One or more yarns become shorter.

Now suppose that that the sequence of edge turnings never stops.
Mark every step as `a` if case a. applies and as `b` if case b. applies.
Our infinite turning is then an infinite sequence of `a`s and `b`s.

In that sequence there cannot be infinitely many `b`s, because every `b` corresponds to
a shortening of the total amount of yarn, and the total length of yarn is finite.

So there must be infinitely many `a`s.
That means that from some point onwards, we see only `a`s and never a `b` anymore.
But every `a` decreases the number of non-up-to-date edges, so a sequence of `a`s is
bound to stop.

Ergo: the sequence cannot be infinite and the computation terminates.

## Lemma 2: No overspinning
**Turning of edges does not cause overspinning: no yarn gets overspun.**

**Proof:**

Suppose none of the yarns is overspun yet, and we turn an edge
$e = (R_j, q_f, q_t)$ on yarns $y_f, y_t$ resulting in yarns $z_f, z_t$.

We assume $y_f$ and $y_t$ are not overspun.

**Case a: Suppose $z_f$ is overspun.**

(i) Then there must be a result text node $s$ of query node $q_f$ in $y_f$ 
that does not reappear in $z_f$.

Because $s$ is in the result, there must be a text node $u$ in the result
of query node $q_t$, such that $sR_ju$.

That means that $u$ belongs to every yarn of $q_t$ that is not overspun.
By hypothesis, $y_t$ is not overspun, so $u$ is member of $y_t$.

By following the definition of turning an edge,
we see that the existence of this $u$ will cause $s$ to be added to $z_f$
which conflicts with (i).

So case a. does not happen.

**Case b: Suppose $z_t$ is overspun.**

(ii) Then there must be a result text node $u$ of query node $q_t$ in $y_t$ 
that does not reappear in $z_t$.

Because $u$ is in the result, there must be a text node $s$ in the result
of query node $q_f$, such that $sR_ju$.

That means that $s$ belongs to every yarn of $q_f$ that is not overspun.
By hypothesis, $y_f$ is not overspun, so $s$ is member of $y_f$.

By following the definition of turning an edge,
we see that the existence of this $s$ will cause $u$ to be added to $z_t$
which conflicts with (ii).

**No more cases: $z_f$ and $z_t$ are not overspun. QED**

## Corollary 3: Empty yarn means no results
**If, after some turning, a yarn becomes empty, the query as a whole has no results.**

**Proof:**

If at some point a yarn, say $y_i$ becomes empty, we know by lemma 2, that it is
still not overspun.
That means that all nodes that occur in a result and that correspond with query node
$q_i$, are contained in this empty yarn $y_i$. 

Hence there are no results, QED.

## Definition: Connected components
A connected component of a graph is a subgraph satisfying two conditions:
* **connectedness:** 
  there is a bridge of edges between each pair of nodes, meaning that for ech pair of
  nodes $q_0$ and $q_{n+1}$ there is a set of nodes
  $q_1, ... , q_n$ in the subgraph such that for each $i$ in $0, ..., n$:
  * there is either an edge from $q_i$ to $q_{i+1}$ or the other way round;
* **maximal:**
  every other subgraph that properly contains this subgraph, does not have the
  **connectedness** property.
  
## Lemma 3: Decomposition
Every graph can be divided into a number of disjunct connected components.

**Proof:**

See the literature on graph theory.

## Lemma 4: Decomposing queries
**The results of a query is essentially the set consisting of the cartesian product of the 
query results of its connected component queries.**

**Proof:**

We prove first:

### Lemma 4a: 
**If a query can be split into two subparts between which there are no edges,
then the results of the whole query can be seen as the cartesian product of 
the results of the two subparts.**

**Proof:**

Say the query is a graph $G$ consisting of
$G_1 = (Q_1, E_1)$ and $G_2 = (Q_2, E_2)$,
with no edges betweem $Q_1$ and $Q_2$ and
with result sets $R_1$ and $R_2$.

Say there are $m_1$ nodes in $Q_1$ and $m_2$ in $Q_2$.

Then $R_1$ is a set of $m_1$ tuples of text nodes,
and $R_2$ is a set of $m_2$ tuples of text nodes.

Every $r_1$ in $R_1$ is a $m_1$-tuple of text nodes satisfying $G_1$.

Every $r_2$ in $R_2$ is a $m_2$-tuple of text nodes satisfying $G_2$.

Then, for every combination of such an $r_1$ and $r_2$,
the concatenation of $r_1$ and $r_2$ a $m_1+m_2$-tuple of text nodes.

The first $m_1$ text nodes instantiate the query nodes of $G_1$
and satisfy all edge constraints of $G_1$. 

They do not have to satisfy the query nodes in $G_2$
in order to be valid for $G$ as a whole,
because they do not correspond to them.

They are not influenced by the constraints of the edges in $G_2$,
because these edges do not reach the nodes of $G_1$.

Analogous for the last $m_2$ nodes in such a combination.

Hence the combination is a result of the whole graph.

Conversely, every result of the whole graph is a tuple that can be decomposed in an $r_1$
which is a result of $G_1$ and an $r_2$ which is a result of $G_2$.

**QED (lemma 4a)**

Now we can prove by induction on the number of connected components of a query
that its results can be seen as the cartesian product 
of the results of its connected components.

**Case a:**

The query consists of just one connected component: the lemma is trivially true.

**Case b:**
The query consists of $n+2$ connected components, $n > 0$.

We assume by way of induction hypothesis
that the lemma holds for all cases up to $n+1$.

Say $G = (G_1 + ... + G_{n+2}, E_1, ... , E_{n+2})$
with result set $R$, 
where each 
$(G_i, E_i)$ is a connected component with result set $R_i$.

Say $H = (G_1 + ... + G_{n+1}, E_1, ... , E_{n+1})$
then we know by induction hypothesis that the results of H are
$R_1 \times ... \times R_{n+1}$.

Now $H$ and $G_{n+2}$ satisfy the conditions of lemma 4a, to the results of

$G = H + (G_{n+2}, E_{n+2})$

is $(R_1 \times ... \times R_{n+1}) \times R_{n+2}$

is $R_1 \times ... \times R_{n+1} \times R_{n+2}$

**QED (lemma 4)**

## Concept: Cyclic queries
A *cycle* in a query is a list of query nodes connected by edges, where the first node is equal to
the last node. 
Here the edges are taken in the directional sense.

### Examples
Here is a graph with a cycle:
$$G = (\{q_1, q_2\}, \{(R, q_1, q_2), (S, q_2, q_1)\})$$
But this is not a cycle:
$$G = (\{q_1, q_2\}, \{(R, q_1, q_2), (S, q_1, q_2)\})$$

## Lemma 5: Eventual full-spinning, with conditions
**For queries consisting of just one connected component:**

**if the query does not contain cycles:**

**if, after some spinning, all edges are up-to-date, then all yarns are fully spun.**

**Proof:**
Because the yarns are never overspun by lemma 2,
we must prove that when all edges are up-to-date,
none of the yarns are *underspun*.

Suppose we are in the situation that all edges are up-to-date.
Suppose that in this situation there is an underspun yarn $y_i$, associated
with query node $q_i$.
That means that $y_i$ contains a text node $s$ that does not belong to any final result.

Let us have a closer look at how $q_i$ lies hooked up in the query graph.
Because the query is a connected component, we have only these possibilities:

**Case a: $q_i$ does not belong to an edge.**

In this case, $q_i$ must the only node, because if there were other nodes,
there would have been bridges from $q_i$ to those other nodes, and hence
$q_i$ would be involved in edges.

So, the query is just one node without edges, hence $y_i$ is the primary yarn,
and hence all nodes in it are results. So this $s$ cannot exist.

**Case b: $q_i$ has an out-going edge or an incoming edge.**

For all $q_i$-outgoing edges $(R_j, q_i, q_t)$ it holds that there
is a $u$ in the yarn $y_t$ of $q_t$ such that $sR_ju$.
This is because all edges are up-to-date.

Like wise, for all $q_i$-incoming edges $(R_k, q_f, q_i)$ it holds that
there is a $u$ in the yarn $y_f$ of $q_f$ such that $uR_ks$.

If all $u$s found in this way belonged to the result, then $s$ would also belong
to the result, because all its edge constraints would be satisfied.

So, if, as we assumed, $s$ is not in the result, then at least one of those $u$s
does not belong to the result.

Then we can repeat the same argument for this $u$, and find a $v$ that does not belong
to the result.

Since there are finitely many nodes, we find a cycle $s_1, ... , s_k$ of text
nodes in yarns $y_1, ... , y_k$ that do not belong to the result.

This contradicts the assumption. QED.

## Example 5b: Cycles can prevent full spinning.
In order to show that the condition of *no cycles* in Lemma 5 cannot be missed,
we show a small example of a graph with a cycle that will not become fully spun
by the algorithm.

$$G = (\{q_1, q_2\}, \{(R, a, c), (R, b, d), S(c, b), S(d, a)\})$$

with primary yarns $y_1, y_2$ for $q_1, q_2$ as follows:

$$y_1 = \{a, b\}$$
$$y_2 = \{c, d\}$$

If we spin the edges $R$ and $S$, nothing happens, because
the elements $a, b \in y_1$ are happily in relationship $R$ with elements $c, d$ in $y_2$,
and
the elements $c, d \in y_2$ are happily in relationship $S$ with elements $a, b$ in $y_2$.

Yet, none of them are part of any result, because there are no results, because the equation

$$xRySx$$

does not have any solutions.

## Analysis
The trouble with a cycle is, that it creates a long distance dependency in the final results
that cannot be "felt" by spinning.

You can also have long distance dependencies by confluence, but these will be eventually solved by just spinning.

For example: if we have to solve both of the following

$$uRvSwTxUk$$
$$aGbHcKmUk$$

then we could solve both chains separately, and if both have a solution, we now that the combined solution is a complete solution.
Hence, if there is no solution, one of the two will have no solution.

In case of the cycle, we are essentially trying to solve

$$xRySz$$
$$x = z$$

where $x = z$ is the long distance dependency.

## Result fetching

Spinning is just part of the solution.
It thins out the space in which solutions exist, and does a better job for graphs without
cycles than for graphs with cycles.

But after spinning, we have to find the valid stitches between the yarns.

That is a separate search process. 
The challenge is to wade through all possibilities in an efficient way.

For graphs without cycles we know that if we start with any text node in any yarn, it will
be part of a result.
And if we start stitching that node to a node in the next yarn, we are sure that the
pair is also part of a result. And so on. So in this case we can stitch arbitrarily (note
that stitching implies that we hop from one yarn to the other by following the appropriate
relationships between text nodes), and always generate results.

In this case, it does not matter much in what order we stitch, because we need to make all stitchings anyway to generate results.

If the graph does have cycles, then we will discover that certain partial stitichings will
lead to no valid results.
Because of that strategy becomes important, because if we are not smart, we could spend a lot
of time in rejecting millions of stitchings, before arriving at the first valid stitiching.

Here is an example, and let us become a bit more concrete.

Suppose our graph has nodes: *Sentence*, *Word1*, *Word2*, all without feature restrictions.
So the primary yarns are all sentences, all words, all words, respectively.
Suppose we have 100,000 sentences, all 10 words, so 1 million words in total.

Suppose our graph has edges:

* word1 is in sentence
* word2 is in sentence
* word1 comes before word2

A solution is a tuple of text nodes $s$, $w_1$, $w_2$, such that $w_1$ is a word
in sentence $s$, $w_2$ is a word in the same sentence $s$, and $w_1$ comes before $w_2$.

Let's start stitching by first picking word1, then word2, then sentence.

We start to pick the first word $w_1$ in the yarn of word1, which is all words>

Then we pick a word $w_2$ in the yarn of word2, such that $w_2$ comes after $w_1$.

Note that we have 999,999 possibilities for $w_2$, of which only the first 10 are part of
a result. All others are not in the same sentence.

So if we try out all these $w_2$s, we get 10 results fairly quickly, and then 999,990 spurious tries, before we try a new node in the yarn of *Word1* and get new results.

By the time we have collected all results,
we have visited a million times on average half a million words, or
$1,000,000 \times 500,000 = 500,000,000,000$ words.

We could have done it in an other way: after picking $w_1$ from the yarn of *Word1*,
we pick a sentence $s_1$ from the yarn of *Sentence* (only one possibility), and from
there we pick a $w_2$ from the yarn of *Word2* such that $w_2$ is in $s_1$ (only 10 possibilities). Of those 10 possibilities, one will be rejected, (where $w_2 = w_1$).

After this, we move to the next $w_1$ in the yarn of $Word1$, which is the second word in the same sentence, pick the same $s_1$ in the sentence yarn, pick the same words in the *Word2*
yarn, reject 2 of them, and deliver 8 results.
And so on.
When we have gone through the whole of $s_1$, we have tried 100 words, and rejected 50 of them. 
And so it goes for all sentences.
In the end we find all 100,000 * 50 = 5,000,000 solutions by visiting 10,000,000 words.

This is a 50,000 fold improvement!

So how can we build a strategy from this observation?

## Stitching strategy

Consider the relationships by which we hop from one yarn to another, while stitching.
Some of them are functional, in the sense that coming from node, they leave only one possibility for the next node in the stitching.

Take for example the relation: *sentence of*. Coming from a word, this relationship leaves
us but one choice: the one sentence of which that word is a part.

Other relationships are virtually the opposite of functional: they leave very many options.

Take for example the relation: *comes after* between words. For any word you have on average the choice of half the total amount of words.

Some relationships that are not functional are still functional in the opposite direction.

Take for example: *contains* between sentences and words. A sentence contains multiple words,
so *contains* is not functional. But going into the other direction, is the precisely the
relation: *sentence of*, which is functional.

Between functional and non-functional relations there is a spectrum of *functionalness*.
Let us call this notion the *spread* of a relation.

### Definition: spread

**The spread of a relation $R$ is the average number of $t$ that satisfies the
equation $fRt$ for each $f$.**

In other words, for every *from* node, compute to how many *to* nodes $R$ brings you.
Take the average, and that is your spread.

## Back to stichting

When stitching, we want to follow the query graph in such a way that we hop from yarn to
yarn by edges with relations with lesser spread first.

This could be a way:

1. compute the spread of all relations in the query graph, and all their converses too
2. for every relation, consider whether it or its converse has the lesser spread
   and if so, replace the relation by its converse in the opposite direction
3. for each node, define the its spread as the sum of the spreads of its outgoing
   relations
4. start with a node that has minimal spread among nodes with outgoing edges
5. go from that node via outgoing edge to a node with minimum spread

In [79]:
contained = 0
embeds = 1
before = 2
after = 3
unequal = 4
overlap = 6

bk = ('book', dict())
ch = ('chapter', dict())
vs = ('verse', dict())
cl = ('clause', dict())
ca = ('clause_atom', dict())
ph = ('phrase', dict())
pa = ('phrase_atom', dict())
w1 = ('word', dict(lex={'JC/', '>JN/'}))
w2 = ('word', dict(sp='verb'))

p2 = ('phrase', dict(det='und'))
w3 = ('word', dict(sp='subs', gn='f', nu='pl'))
w4 = ('word', dict(sp='adjv', gn='f', nu='pl'))

c2 = ('clause', dict())
p3 = ('phrase', dict())
p4 = ('phrase', dict())
w5 = ('word', dict())
w6 = ('word', dict())
w7 = ('word', dict())

nodes1 = [bk, ch, vs, cl, ca, ph, pa, w1, w2]
edges1 = [
    [1, contained, 0],
    [2, contained, 1],
    [3, contained, 2],
    [4, contained, 3],
    [5, contained, 4],
    [6, contained, 5],
    [7, contained, 6],
    [8, after, 7],
    [8, contained, 3],
]

nodes2 = [p2, w3, w4]
edges2 = [
    [1, contained, 0],
    [2, contained, 0],
    [1, before, 2],
]
nodes3 = [p3, p4, w5, w6, w7, c2]
edges3 = [
    [2, contained, 0],
    [4, contained, 0],
    [3, contained, 1],
    [2, before, 3],
    [3, before, 4],
    [0, contained, 5],
    [1, contained, 5],
    [0, unequal, 1],
]

In [80]:
S.runSearch(nodes3, edges3)

  0.00s Search template OK: 6 search nodes and 8 search edges
plan with 6 nodes and 8 edges
node  0-phrase        (253174 in yarn)
node  1-phrase        (253174 in yarn)
node  2-word          (426581 in yarn)
node  3-word          (426581 in yarn)
node  4-word          (426581 in yarn)
node  5-clause        ( 88000 in yarn)
edge  4-word          = in  =>  0-phrase        @       1
edge  0-phrase        = in  =>  5-clause        @       1
edge  0-phrase        = has =>  2-word          @     2.4
edge  5-clause        = has =>  1-phrase        @       3
edge  0-phrase        = <>  =>  1-phrase        @2.53e+05
edge  1-phrase        = has =>  3-word          @     1.8
edge  3-word          = <<  =>  4-word          @1.69e+05
edge  3-word          = >>  =>  2-word          @ 2.3e+05
  6.98s Search prepared: total amount of yarn: 1874091


In [81]:
S.legend

('phrase', 'phrase', 'word', 'word', 'word', 'clause')

In [82]:
i = 0
for s in S.results():
    i += 1
    print(s)
    if i == 10: break

(605793, 605794, 1159, 1160, 1161, 426799)
(605793, 605794, 1159, 1160, 1162, 426799)
(605793, 605794, 1159, 1160, 1163, 426799)
(605793, 605794, 1159, 1160, 1164, 426799)
(606150, 606151, 1720, 1721, 1722, 426921)
(606150, 606151, 1720, 1721, 1723, 426921)
(607746, 607747, 4819, 4824, 4825, 427418)
(607746, 607747, 4819, 4821, 4825, 427418)
(607746, 607747, 4819, 4822, 4825, 427418)
(607746, 607747, 4819, 4823, 4825, 427418)


In [83]:
i = 0
seen = set()
indent(reset=True)
info('fetching results')
for s in S.results():
    if s[3] not in seen:
        seen.add(s[3])
        if len(seen) <= 10:
            print(s)
    i += 1
info('{} condensed to {} results'.format(i, len(seen)))

  0.00s fetching results
(605793, 605794, 1159, 1160, 1161, 426799)
(606150, 606151, 1720, 1721, 1722, 426921)
(607746, 607747, 4819, 4824, 4825, 427418)
(607746, 607747, 4819, 4821, 4825, 427418)
(607746, 607747, 4819, 4822, 4825, 427418)
(607746, 607747, 4819, 4823, 4825, 427418)
(608322, 608323, 5803, 5805, 5807, 427601)
(608322, 608323, 5803, 5806, 5807, 427601)
(608369, 608370, 5868, 5869, 5871, 427616)
(608369, 608370, 5868, 5870, 5871, 427616)
 1m 40s 7699 condensed to 751 results


In [77]:
info('Getting gapped phrases')
results = []
for c in F.otype.s('clause'):
    ps = L.d(c, 'phrase')
    for p in ps:
        words = L.d(p, 'word')
        (bp, ep) = (words[0], words[-1])
        for q in ps:
            if p == q: continue
            bq = L.d(q, 'word')[0]
            if bp < bq and bq < ep:
                results.append((p, q, bp, bq, ep, c))
info('{} results'.format(len(results)))
for r in results[0:10]:
    print(r)

    20s Getting gapped phrases
    25s 373 results
(605793, 605794, 1159, 1160, 1164, 426799)
(606150, 606151, 1720, 1721, 1723, 426921)
(607746, 607747, 4819, 4821, 4828, 427418)
(608322, 608323, 5803, 5805, 5809, 427601)
(608369, 608370, 5868, 5869, 5875, 427616)
(608705, 608706, 6515, 6521, 6530, 427723)
(609286, 609287, 7431, 7432, 7437, 427917)
(609997, 609998, 8502, 8507, 8520, 428159)
(609997, 609999, 8502, 8508, 8520, 428159)
(610379, 610380, 9127, 9129, 9133, 428286)


In [251]:
import re
escapes = (
    '\\\\',
    '\\ ',
    '\\t',
    '\\n',
    '\\|',
    '\\=',
)
def esc(x):
    for (i, c) in enumerate(escapes):
        x = x.replace(c, chr(i))
    return x
def unesc(x):
    for (i, c) in enumerate(escapes):
        x = x.replace(chr(i), c[1])
    return x

In [341]:
def syntax(search):
    lines = search.split('\n')
    tokens = tokenize(lines)
    if tokens != None:
        for t in tokens:
            print(t)
        return tokens
    else:
        for (i, line) in enumerate(lines):
            print('{:>2} {}'.format(i, line))
        return None

def tokenize(lines):            
    operators = '''
    = # < > 
    == && ## || [[ ]] << >>
'''.strip().split()
    edges = '''
    -\S+> <\S+-
'''.strip().split()

    opPat = '|'.join(
        tuple(o.\
                replace('&', '\\&').\
                replace('|', '\\|').\
                replace('[', '\\[').\
                replace(']', '\\]')\
            for o in operators
        )+\
        tuple(edges)
    )
    atomPat = '(\s*)([^ \t=]+)(?:(?:\s*\Z)|(?:\s+(.*)))$'
    namePat = '[A-Za-z0-9_-]+'
    atomRe = re.compile(atomPat)
    relPat = '^\s*({nm})\s+({op})\s+({nm})\s*$'.format(nm=namePat, op=opPat)
    
    nameRe = re.compile('^{}$'.format(namePat))
    relRe = re.compile(relPat)
    whiteRe = re.compile('^\s*$')
    
    def getFeatures(x):
        features = {}
        wrongs = []
        featureList = (x if x != None else '').split()
        for feat in featureList:
            featComps = feat.split('=', 1)
            if len(featComps) != 2:
                wrongs.append(unesc(feat))
                continue
            featName = unesc(featComps[0])
            featValList = featComps[1]
            featVals = set(unesc(featVal) for featVal in featValList.split('|'))
            features[featName] = featVals
        return (features, wrongs)
    
    tokens = []
    allGood = True
    for (i, line) in enumerate(lines):
        if line.startswith('#') or whiteRe.match(line): continue
        good = False
        for x in [True]:
            match = relRe.match(line)
            if match:
                tokens.append((i, 'rel', match.group(1), match.group(2), match.group(3)))
                good = True
                break
            match = atomRe.match(esc(line))
            if match:
                (indent, atom, features) = match.groups()
                atomComps = atom.split(':', 1)
                if len(atomComps) == 1:
                    name = ''
                else:
                    name = unesc(atomComps[0])
                    atom = unesc(atomComps[1])
                    mt = nameRe.match(name)
                    if not mt:
                        error('Illegal name at line {}: "{}"'.format(i, name))
                        good = False
                (features, wrongs) = getFeatures(features)
                if len(wrongs):
                    for wrong in wrongs:
                        error('Illegal feature specification at line {}: "{}"'.format(i, wrong))
                    good = False
                    break                
                tokens.append((i, 'atom', len(indent), name, atom, features))
                good = True
                break
            (features, wrongs) = getFeatures(esc(line))
            if len(wrongs):
                for wrong in wrongs:
                    error('Illegal feature specification at line {}: "{}"'.format(i, wrong))
                good = False
                break                
            tokens.append((i, 'feat', features))
            good = True
            break
        if not good: allGood = False
    return tokens if allGood else None

In [346]:
def semantics(tokens):
    prevKind = None
    good = True
    meaning = []
    qnames = {}
    qnodes = []
    qedges = []
    edgeLine = {}
    nodeLine = {}
    tokens = sorted(tokens, key=lambda t: (len(tokens)+t[0]) if t[1] == 'rel' else t[0])

    # atomStack is a stack of qnodes with there indent levels
    # such that every next member is one level deeper
    # and every member is the last qnode encountered at that level
    # The stack is implemented as a dict, keyed by the indent, and valued by the qnode
    atomStack = {}
    
    for (i, kind, *fields) in tokens:
        if kind == 'atom':
            (indent, name, otype, features) = fields
            qnodes.append((otype, features))
            q = len(qnodes) - 1
            nodeLine[q] = i
            name = ':{}'.format(i) if name == '' else name
            qnames[name] = q
            if len(atomStack) == 0:
                if indent > 0:
                    error('Unexpected indent at line {}: {}, expected {}'.format(i, indent, 0))
                    good = False
                atomStack[0] = q
            else:
                atomNest = sorted(atomStack.items(), key=lambda x: x[0])
                top = atomNest[-1]
                if indent == top[0]:  
                    # sibling of previous atom
                    if len(atomNest) > 1:
                        # take the qnode of the subtop of the atomStack, if there is one
                        qedges.append((q, ']]', atomNest[-2][1]))
                        edgeLine[len(qedges) - 1] = i
                elif indent > top[0]:
                    # child of previous atom
                    qedges.append((q, ']]', top[1]))
                    edgeLine[len(qedges) - 1] = i
                else:
                    # outdent action: look up the proper parent in the stack
                    if indent not in atomStack:
                        # parent cannot be found: indentation error
                        error('Unexpected indent at line {}: {}, expected one of {}'.format(
                            i, indent,
                            ','.join(str(at[0]) for at in atomNest if at[0] < indent),
                        ))
                        good = False
                    else:
                        parents = [at[1] for at in atomNest if at[0] < indent]
                        if len(parents) != 0: # if not already at outermost level
                            qedges.append((q, ']]', parents[-1]))
                            edgeLine[len(qedges) - 1] = i
                        removeKeys = [at[0] for at in atomNest if at[0] > indent]
                        for rk in removeKeys: del atomStack[rk]
                atomStack[indent] = q
        elif kind == 'feat':
            features = fields[0]
            if prevKind != None and prevKind != 'atom':
                error('Features without atom at line {}: "{}"'.format(i, features))
                good = False
            else:
                qnodes[-1][1].update(features)
        elif kind == 'rel':
            (fName, opName, tName) = fields
            f = qnames.get(fName, None)
            t = qnames.get(tName, None)
            namesGood = True
            for (q, n) in ((f, fName), (t, tName)):
                if q == None:
                    error('Relation with undefined name at line {}: "{}"'.format(i, n))
                    namesGood = False
            if not namesGood:
                good = False
            else:
                qedges.append((f, opName, t))                
                edgeLine[len(qedges) - 1] = i
        prevKind = kind
    if not good:
        return None
    return (qnames, qnodes, qedges, nodeLine, edgeLine)

In [347]:
def parse(search):
    tokens = syntax(search)
    if tokens == None:
        return None
    meaning = semantics(tokens)
    if meaning == None:
        for (i, line) in enumerate(search.split('\n')):
            print('{:>2} {}'.format(i, line))
        return None
    return meaning

In [351]:
parse(search1)

(2, 'atom', 0, '', 'clause', {})
(4, 'atom', 4, 'p1', 'phrase', {'det': {'und'}})
(5, 'atom', 8, 'w1', 'word', {'lex': {'WS[', 'JC/'}})
(6, 'feat', {'gn': {'f'}, 'ps': {'p1', 'p2', 'p3'}})
(7, 'atom', 8, 'w3', 'word', {})
(8, 'rel', 'w1', '<', 'w3')
(10, 'atom', 4, 'p2', 'phrase', {})
(11, 'atom', 8, 'w2', 'word', {})
(12, 'rel', 'w1', '<', 'w2')
(13, 'rel', 'w2', '<', 'w3')
(15, 'rel', 'p2', '<mother-', 'p1')


({':2': 0, 'p1': 1, 'p2': 4, 'w1': 2, 'w2': 5, 'w3': 3},
 [('clause', {}),
  ('phrase', {'det': {'und'}}),
  ('word', {'gn': {'f'}, 'lex': {'JC/', 'WS['}, 'ps': {'p1', 'p2', 'p3'}}),
  ('word', {}),
  ('phrase', {}),
  ('word', {})],
 [(1, ']', 1, 0),
  (2, ']', 1, 1),
  (3, ']', 1, 1),
  (4, ']', 1, 0),
  (5, ']', 1, 4),
  (2, '<', 1, 3),
  (2, '<', 1, 5),
  (5, '<', 1, 3),
  (4, 'mother', -1, 1)],
 {0: 2, 1: 4, 2: 5, 3: 7, 4: 10, 5: 11},
 {0: 4, 1: 5, 2: 7, 3: 10, 4: 11, 5: 8, 6: 12, 7: 13, 8: 15})